## WIKIPEDIA CONVERSATIONAL AGENT

In [13]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [14]:
from langchain.tools import tool

### WIKIPEDIA LIBRARY

- python's wikipedia library wraps official wikipedia API called MediaWiki API (https://www.mediawiki.org/wiki/API) 
- installation by Wikipedia API
- wikipedia.summary(title, sentences) - number of sentences in result, returns the result in string format
- wikipedia.search(title, result) - number of results, returns list of titles
- wikipedia.page(title) - this method is used to get content, categories, coordinates, images, links and metadata -> returns WikipediaPage object
- "summary" attribute of page() is used to obtain the plain text summary of that particular topic

In [15]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [16]:
tools = [search_wikipedia]

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [18]:
functions = [format_tool_to_openai_function(f) for f in tools]

model = ChatOpenAI(temperature=0).bind(functions=functions)

In [19]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [37]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly wikipedia bot that can tell details and answer questions about anything the user asks using information from wikipedia"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [38]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [39]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [40]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [41]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Hello Bob! How can I assist you today?'}

In [42]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you today, Bob?

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?')],
 'output': 'Your name is Bob. How can I assist you today, Bob?'}

In [43]:
agent_executor.invoke({"input": "what is artemis 1 mission?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Artemis I mission'}`


Page: List of Artemis missions
Summary: The Artemis program is a human spaceflight program by the United States. The Artemis program is intended to reestablish a human presence on the Moon for the first time since Apollo 17 in 1972; mid-term objectives include establishing an international expedition team, and a sustainable human presence on the Moon. Long-term objectives for Artemis are laying the foundations for the extraction of lunar resources, and eventually making crewed missions to Mars and beyond feasible.
To date, missions in the program are aimed at exploration of the Moon, including crewed and robotic exploration of the lunar surface. Three flights of the Orion Multi-Purpose Crew Vehicle are currently planned for launch in the Artemis program in the early 2020s, beginning with Artemis 1. Before Artemis was named, the flights were referred to as "Orion missions". Nume

{'input': 'what is artemis 1 mission?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='what is artemis 1 mission?'),
  AIMessage(content="Artemis 1, officially known as Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission that marked NASA's return to lunar exploration after the conclusion of the Apollo program. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket. The main objective of Artemis 1 was to test the Orion spacecraft, especially its heat shield, in preparation for subsequent Artemis missions. The mission aimed to reestablish a human presence on the Moon and demonstrate technologies needed for future scientific studies, including exploration of Mars.\n\nArtemis 1 was launched on No

In [44]:
agent_executor.invoke({"input": "when was it launched?"})



> Entering new AgentExecutor chain...
Artemis 1, also known as Exploration Mission-1 (EM-1), was launched on November 16, 2022.

> Finished chain.


{'input': 'when was it launched?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='what is artemis 1 mission?'),
  AIMessage(content="Artemis 1, officially known as Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission that marked NASA's return to lunar exploration after the conclusion of the Apollo program. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket. The main objective of Artemis 1 was to test the Orion spacecraft, especially its heat shield, in preparation for subsequent Artemis missions. The mission aimed to reestablish a human presence on the Moon and demonstrate technologies needed for future scientific studies, including exploration of Mars.\n\nArtemis 1 was launched on Novembe

In [45]:
agent_executor.invoke({"input": "what is the next phase in the mission?"})



> Entering new AgentExecutor chain...
The next phase in the Artemis mission series is Artemis 2. Artemis 2 will be a crewed mission that follows the uncrewed Artemis 1 mission. The main objective of Artemis 2 is to send astronauts on a mission around the Moon. This mission will serve as a test for the Orion spacecraft and the Space Launch System (SLS) rocket with a crew on board. Artemis 2 is scheduled for launch no earlier than September 2025. Following Artemis 2, the Artemis 3 mission is planned to be the first crewed lunar landing mission since the Apollo program.

> Finished chain.


{'input': 'what is the next phase in the mission?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='what is artemis 1 mission?'),
  AIMessage(content="Artemis 1, officially known as Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission that marked NASA's return to lunar exploration after the conclusion of the Apollo program. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket. The main objective of Artemis 1 was to test the Orion spacecraft, especially its heat shield, in preparation for subsequent Artemis missions. The mission aimed to reestablish a human presence on the Moon and demonstrate technologies needed for future scientific studies, including exploration of Mars.\n\nArtemis 1 was la

In [46]:
agent_executor.invoke({"input": "when is that happening?"})



> Entering new AgentExecutor chain...
Artemis 2, the crewed mission around the Moon, is scheduled for launch no earlier than September 2025. The exact date may be subject to change based on mission planning and development.

> Finished chain.


{'input': 'when is that happening?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='what is artemis 1 mission?'),
  AIMessage(content="Artemis 1, officially known as Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission that marked NASA's return to lunar exploration after the conclusion of the Apollo program. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket. The main objective of Artemis 1 was to test the Orion spacecraft, especially its heat shield, in preparation for subsequent Artemis missions. The mission aimed to reestablish a human presence on the Moon and demonstrate technologies needed for future scientific studies, including exploration of Mars.\n\nArtemis 1 was launched on Novem

In [47]:
agent_executor.invoke({"input": "where is the artemis 2 going to land on moon?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Artemis 2'}`


Page: Artemis 2
Summary: Artemis 2 (officially Artemis II) is a scheduled mission of the NASA-led Artemis program. It will use the second launch of the Space Launch System (SLS) and include the first crewed mission of the Orion spacecraft. The mission is scheduled for no earlier than September 2025. Four astronauts will perform a flyby of the Moon and return to Earth, becoming the first crew to travel beyond low Earth orbit since Apollo 17 in 1972. Artemis 2 will be the first crewed launch from Launch Complex 39B of the Kennedy Space Center since STS-116 in 2006.
Originally designated Exploration Mission-2 (EM-2), the mission was intended to collect samples from a captured asteroid in lunar orbit by the now-cancelled robotic Asteroid Redirect Mission; it was renamed after the introduction of the Artemis program.

Page: Artemis program
Summary: The Artemis program is a Moon exploration 

{'input': 'where is the artemis 2 going to land on moon?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='what is artemis 1 mission?'),
  AIMessage(content="Artemis 1, officially known as Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission that marked NASA's return to lunar exploration after the conclusion of the Apollo program. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket. The main objective of Artemis 1 was to test the Orion spacecraft, especially its heat shield, in preparation for subsequent Artemis missions. The mission aimed to reestablish a human presence on the Moon and demonstrate technologies needed for future scientific studies, including exploration of Mars.\n\nArtemis 1

### Create a chatbot

In [48]:
tools = [search_wikipedia]

In [ ]:
import panel as pn  # for creating interactive web-based dashboards 
pn.extension()
import panel as pn
import param # provides parameters which are useful for creating dynamic objects that can interact with UI

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    
    """
    `convchain` is invoked when the user submits a query. it processes the input query 
     using the `qa` chain, stores the chatbot's response and updates the UI with the user's query 
     and the chatbot's response. conversation history is updated with each interaction. 
    """   
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 
        


In [ ]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')  # widget for user input where users can type their queries

"""binds the `convchain` method to the TextInput widget, so that chatbot can process 
the query each time the user submits text."""
conversation = pn.bind(cb.convchain, inp)   

# this defines the layout for conversation tab 
tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

# Main layout of the application
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard